In [1]:
import torch, os
import pandas as pd
import numpy as np
from transformers import FunnelTokenizerFast, DataCollatorWithPadding, FunnelForSequenceClassification
from ProjectDict import *
from tqdm import tqdm

device = "cpu"
if torch.cuda.is_available():
    device = "cuda"
# device = 'dml' if torch.cuda.is_available() else 'cpu'
print(f"device: {device}")
print(torch.version.hip)

device: cuda
None


In [2]:
rpath = os.path.join("inspected_data")
theme = wonder_boy # laughing_man | wonder_boy
theme_en = wonder_boy_en # laughing_man_en | wonder_boy_en
model_type = model_types.base # model_types.base | model_types.fine_tuned
df = pd.read_csv(os.path.join(rpath, f"{theme}_splitted.csv"),)

In [3]:
rows = []
for row in df.to_numpy():
    text = ""
    for sentence in row:
        if type(sentence) == str:
            text += '' + sentence
    rows.append(text)

In [4]:
arr = df.to_numpy()
len(arr)

125

In [5]:
tokenizer_funnel = FunnelTokenizerFast.from_pretrained("kykim/funnel-kor-base")
if model_type == model_types.base:
    model_funnel = FunnelForSequenceClassification.from_pretrained("kykim/funnel-kor-base")
else:
    model_funnel = FunnelForSequenceClassification.from_pretrained("./FunnelLM/")
model_funnel = model_funnel.to(device)

model_funnel.eval()

Some weights of the model checkpoint at kykim/funnel-kor-base were not used when initializing FunnelForSequenceClassification: ['decoder.layers.0.ffn.layer_norm.bias', 'decoder.layers.1.attention.v_head.bias', 'decoder.layers.1.attention.r_r_bias', 'decoder.layers.1.attention.post_proj.bias', 'decoder.layers.0.ffn.linear_2.weight', 'decoder.layers.0.ffn.linear_1.bias', 'decoder.layers.1.ffn.layer_norm.bias', 'decoder.layers.1.ffn.linear_1.bias', 'decoder.layers.0.attention.seg_embed', 'decoder.layers.0.attention.v_head.bias', 'decoder.layers.0.attention.r_w_bias', 'decoder.layers.0.attention.post_proj.weight', 'decoder.layers.0.ffn.linear_2.bias', 'decoder.layers.1.ffn.layer_norm.weight', 'decoder.layers.1.ffn.linear_2.weight', 'decoder.layers.0.attention.v_head.weight', 'decoder.layers.1.attention.r_s_bias', 'decoder.layers.0.attention.r_r_bias', 'decoder.layers.0.ffn.layer_norm.weight', 'decoder.layers.1.attention.layer_norm.weight', 'decoder.layers.1.attention.layer_norm.bias', 'dec

FunnelForSequenceClassification(
  (funnel): FunnelBaseModel(
    (embeddings): FunnelEmbeddings(
      (word_embeddings): Embedding(42000, 768)
      (layer_norm): LayerNorm((768,), eps=1e-09, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): FunnelEncoder(
      (attention_structure): FunnelAttentionStructure(
        (sin_dropout): Dropout(p=0.1, inplace=False)
        (cos_dropout): Dropout(p=0.1, inplace=False)
      )
      (blocks): ModuleList(
        (0-2): 3 x ModuleList(
          (0-5): 6 x FunnelLayer(
            (attention): FunnelRelMultiheadAttention(
              (hidden_dropout): Dropout(p=0.1, inplace=False)
              (attention_dropout): Dropout(p=0.1, inplace=False)
              (q_head): Linear(in_features=768, out_features=768, bias=False)
              (k_head): Linear(in_features=768, out_features=768, bias=True)
              (v_head): Linear(in_features=768, out_features=768, bias=True)
              (post_pro

In [6]:
print(model_funnel)

FunnelForSequenceClassification(
  (funnel): FunnelBaseModel(
    (embeddings): FunnelEmbeddings(
      (word_embeddings): Embedding(42000, 768)
      (layer_norm): LayerNorm((768,), eps=1e-09, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): FunnelEncoder(
      (attention_structure): FunnelAttentionStructure(
        (sin_dropout): Dropout(p=0.1, inplace=False)
        (cos_dropout): Dropout(p=0.1, inplace=False)
      )
      (blocks): ModuleList(
        (0-2): 3 x ModuleList(
          (0-5): 6 x FunnelLayer(
            (attention): FunnelRelMultiheadAttention(
              (hidden_dropout): Dropout(p=0.1, inplace=False)
              (attention_dropout): Dropout(p=0.1, inplace=False)
              (q_head): Linear(in_features=768, out_features=768, bias=False)
              (k_head): Linear(in_features=768, out_features=768, bias=True)
              (v_head): Linear(in_features=768, out_features=768, bias=True)
              (post_pro

In [7]:
model_funnel.classifier = torch.nn.Identity()

In [8]:
pr_in = []
for str_arr in arr:
    str_arr = [string for string in str_arr if type(string) == str]
    pr_in.append(' '.join(str_arr))
len(pr_in)

125

In [9]:
collator = DataCollatorWithPadding(tokenizer_funnel, padding=True, max_length=tokenizer_funnel.model_max_length)

In [10]:
tokenized_in = tokenizer_funnel(pr_in, padding=True, truncation=True, max_length=tokenizer_funnel.model_max_length)
collated_in = collator(tokenized_in)
collated_in['input_ids']

You're using a FunnelTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
c:\Users\sizzf\.conda\envs\torch\lib\site-packages\transformers\tokenization_utils_base.py:2382: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


tensor([[    2, 39672,  8214,  ...,     0,     0,     0],
        [    2, 31872, 19397,  ...,     0,     0,     0],
        [    2,  2011, 23010,  ...,     0,     0,     0],
        ...,
        [    2, 25341, 40661,  ...,     0,     0,     0],
        [    2, 31872, 19397,  ...,     0,     0,     0],
        [    2, 31872, 19397,  ...,     0,     0,     0]])

In [11]:
len(collated_in['input_ids'][0]), len(pr_in)

(512, 125)

In [12]:
batch_size = 32
N = len(pr_in)
embeddings = []

with torch.no_grad():
    for n in tqdm(range(0+batch_size, N+batch_size-1, batch_size)):
        start, end = max(0, n-batch_size), n
        input_ids = collated_in['input_ids'][start:end].to(device)
        attention_mask = collated_in['attention_mask'][start:end].to(device)
        token_type_ids = collated_in['token_type_ids'][start:end].to(device)
        embedding = model_funnel.forward(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids).logits
        # print(f"{n}: {embedding}")
        embeddings.append(embedding)
        del input_ids
        del attention_mask
        del token_type_ids
        del embedding


100%|██████████| 4/4 [00:03<00:00,  1.02it/s]


In [13]:
embeddings = torch.concat(embeddings, dim=0)

In [14]:
torch.save(embeddings, f"cache/{theme_en}_embeddings_{model_type}.pt")